# Handwritten Digit Recognition

This is a model for recognizing a handwritten digit from 0 to 9


## Importing Packages

Imports the below listed libraries

- Numpy 
- Matplotlib
- TensorFlow
- Scikit learn

In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score